<a href="https://colab.research.google.com/github/mmsamiei/just-practice-deep/blob/master/Abbas_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchtext
from torchtext.data import Field
import spacy

from spacy.symbols import ORTH
my_tok = spacy.load('en')

def spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

QUERY = Field(lower=True, tokenize=spacy_tok)
RESPONSE = Field(lower=True, tokenize=spacy_tok, is_target=True, init_token='<bos>', eos_token='<eos>')

In [0]:
import torch
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [0]:
my_tok.tokenizer.add_special_case("don't", [{ORTH: "do"}, {ORTH: "n't"}])

In [0]:
from torchtext.data import TabularDataset

train_dataset = TabularDataset(path="./formatted_movie_lines.txt", format="CSV",
                               fields=[("query", QUERY),("response", RESPONSE)],
                               csv_reader_params={"delimiter":'\t'})

In [5]:
QUERY.build_vocab(train_dataset)
RESPONSE.build_vocab(train_dataset)
print(QUERY.vocab.stoi['film'])
print(QUERY.vocab.itos[33])
print(len(QUERY.vocab))
print(len(RESPONSE.vocab))

1059
're
48505
49036


In [0]:
from torchtext.data import BucketIterator, interleave_keys

batch_size = 32

def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    # (new example to add, current effective batch size, current count of examples in the batch) 
    # when returned value meets batch_size (effective, innate effective batch_size 
    # defined as global bala bala) then wraper create a batch

    sum_len = len(new.query) + len(new.response)

    if sum_len > 500:
      return batch_size
    elif sum_len > 300: 
      return sofar + 16
    elif sum_len > 200: 
      return sofar + 8
    elif sum_len > 120: 
      return sofar + 4
    elif sum_len > 60: 
      return sofar + 2
    elif sum_len > 30: 
      return sofar + 1
    elif sum_len > 20: 
      return sofar + 0.5
    else:
      return sofar + 0.25
    
    
       

train_iterator = BucketIterator(dataset= train_dataset, batch_size=batch_size,
                                batch_size_fn = batch_size_fn
                                ,device=device
                                ,sort_key=lambda x: interleave_keys(len(x.query), len(x.response))
                                , sort = True
                                , repeat = False)

In [68]:
for batch in (iter(train_iterator)):
  print(batch.query.shape, "\t", batch.response.shape)

torch.Size([3, 128]) 	 torch.Size([5, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([4, 128])
torch.Size([2, 128]) 	 torch.Size([5, 128])
torch.Size([2, 128]) 	 torch.Size([5, 128])
torch.Size([2, 128]) 	 torch.Size([5, 128])
torch.Size([2, 128]) 	 torch.Size([5, 128])
torch.Size([2, 128]) 	 torch.Size([5, 128])
torch.Size([2, 128]) 	 torch.Size([5, 128])
torch.Size([3, 128]) 	 torch.Size([5, 128])
torch.Size([3, 128]) 	 torch.Size([4, 128])
torch.Size([3, 128]) 	 torch.Size([4, 128])
torch.Size([3, 128]) 	 torch.Siz

In [0]:
import torch.nn as nn
class Bradley(nn.Module):
  def __init__(self, src_voc_sze, trg_voc_sze, hid_sze, num_head, num_enc, num_dec):
    super(Bradley, self).__init__()
    self.hid_sze = hid_sze
    self.src_word_embedding = nn.Embedding(src_voc_sze, self.hid_sze)
    self.trg_word_embedding = nn.Embedding(trg_voc_sze, self.hid_sze)
    self.trg_pos_embedding = nn.Embedding(1000, self.hid_sze)
    self.num_head = num_head
    self.transformer = nn.Transformer(self.hid_sze, self.num_head, num_enc, num_dec)
    self.fc = nn.Linear(self.hid_sze, trg_voc_sze)
  
  def forward(self, src, trg):
    #src = [src sent len, batch_size]
    #trg = [trg sent len, batch_size]
    temp_src = self.src_word_embedding(src)
    temp_trg = self.trg_word_embedding(trg)
    trg_sent_len, batch_size = trg.shape[0], trg.shape[1]
    trg_pos = self.trg_pos_embedding(torch.arange(0, trg_sent_len).unsqueeze(0).
                                     repeat(batch_size,1).to(device)).transpose(0,1)
    trg_mask = self._generate_square_subsequent_mask(trg_sent_len)
    temp_trg = temp_trg + trg_pos
    temp = self.transformer(temp_src, temp_trg, tgt_mask=trg_mask)
    return self.fc(temp)
  
  def _generate_square_subsequent_mask(self, sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    mask = mask.to(device)
    return mask


  def greedy_inference_one_sample(self, src, max_len=20):

    self.eval()
    with torch.no_grad():
      #src = [src sent len]
      src_len = src.shape[0]
      src = src.unsqueeze(1)
      #src = [sent_len, 1]
      trg = src.new_full((1,1), RESPONSE.vocab.stoi['<bos>'])
      #trg = [1,1]

      translation_step = 0
      while translation_step < max_len:
        out = self.forward(src, trg)
        out = out[-1,:]
        #out = [batch_size, trg_vocab_size]
        nex = out.argmax(dim=1).unsqueeze(0)
        #nex = [1, 1]
        trg = torch.cat((trg, nex), dim=0)
        translation_step += 1
    return trg



In [0]:
bradley_model = Bradley(src_voc_sze=len(QUERY.vocab), trg_voc_sze=len(RESPONSE.vocab),
                        hid_sze=128, num_head=4,
                        num_enc=4, num_dec=3)
bradley_model = bradley_model.to(device)

In [0]:
from tqdm import tqdm

criterion = nn.CrossEntropyLoss(ignore_index=RESPONSE.vocab.stoi['<pad>'])
lr = 0.5
optimizer = torch.optim.SGD(bradley_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
epoch_number = 5

print(len(train_dataset)/batch_size)

for epoch in range(1, epoch_number+1):
  print("epoch ", epoch)
  bradley_model.train()
  for i, batch in enumerate(iter(train_iterator)):
    src = batch.query
    trg = batch.response
    optimizer.zero_grad()
    out = bradley_model(src, trg)
    loss = criterion(out[:-1,:].view(-1, out.shape[2]), trg[1:,:].view(-1))
    if(i%50==0):
      print("{}: {}".format(i, loss.item()))
    print('current memory allocated: {}'.format(torch.cuda.memory_allocated() / 1024 ** 2))
    print(src.shape)
    print(trg.shape)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(bradley_model.parameters(), 1)
    optimizer.step()
    torch.cuda.empty_cache()
    del loss
    del src
    del trg
    del out


6915.0625
epoch  1
0: 10.818584442138672
current memory allocated: 8129.85546875
torch.Size([3, 128])
torch.Size([5, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8146.77880859375
torch.Size([2, 128])
torch.Size([4, 128])
current memory allocated: 8

In [61]:
#source_sentence = train_dataset[3].query
source_sentence = ['who','is','joker', '?']
print(source_sentence)

['who', 'is', 'joker', '?']


In [62]:
x = QUERY.numericalize([source_sentence]).to(device)
x = x.flatten()
print(x.shape)
result = bradley_model.greedy_inference_one_sample(x)

torch.Size([4])


In [63]:
result

tensor([[  2],
        [  6],
        [ 66],
        [  6],
        [ 66],
        [  6],
        [ 66],
        [  6],
        [ 66],
        [  6],
        [ 35],
        [ 12],
        [125],
        [  4],
        [  3],
        [  4],
        [  3],
        [  4],
        [  3],
        [  4],
        [  3]], device='cuda:0')

In [64]:
print(RESPONSE.vocab.itos[35])
print(RESPONSE.vocab.itos[12])
print(RESPONSE.vocab.itos[125])
print(RESPONSE.vocab.itos[6])
print(RESPONSE.vocab.itos[66])
print(RESPONSE.vocab.itos[6])
print(RESPONSE.vocab.itos[66])
print(RESPONSE.vocab.itos[125])


was
a
little
i
think
i
think
little


In [0]:
def batch_index_to_strings(trg):
  # trg = [sent_len, batch_size]
  temp = trg.transpose(0,1)
  for i, row in enumerate(temp):
    print(row)

batch_index_to_strings(result)

In [0]:
mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

In [0]:
mask = (torch.triu(torch.ones(5, 5)) == 1).transpose(0, 1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

In [0]:
mask